In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [2]:
url_inicial = 'http://revistamedica.imss.gob.mx/numero_actual'
url_root = 'http://revistamedica.imss.gob.mx/numero_actual'
r=requests.get(url_inicial)

In [3]:
soup = BeautifulSoup(r.text, 'html.parser')

In [4]:
box = soup.find('div', class_='region region-content')
volumen=box.findAll('div', class_='row text-justify')
vol = [x.find('a').get('href')for x in volumen]
vol=[urljoin(url_root,i) for i in vol]
vol

['http://revistamedica.imss.gob.mx/editorial/index.php/revista_medica/article/view/4627',
 'http://revistamedica.imss.gob.mx/editorial/index.php/revista_medica/article/view/4245',
 'http://revistamedica.imss.gob.mx/editorial/index.php/revista_medica/article/view/4409',
 'http://revistamedica.imss.gob.mx/editorial/index.php/revista_medica/article/view/4425',
 'http://revistamedica.imss.gob.mx/editorial/index.php/revista_medica/article/view/4446',
 'http://revistamedica.imss.gob.mx/editorial/index.php/revista_medica/article/view/4478',
 'http://revistamedica.imss.gob.mx/editorial/index.php/revista_medica/article/view/4487',
 'http://revistamedica.imss.gob.mx/editorial/index.php/revista_medica/article/view/4494',
 'http://revistamedica.imss.gob.mx/editorial/index.php/revista_medica/article/view/4523',
 'http://revistamedica.imss.gob.mx/editorial/index.php/revista_medica/article/view/4532',
 'http://revistamedica.imss.gob.mx/editorial/index.php/revista_medica/article/view/4546',
 'http://r

In [5]:
def get_url_items(sopa, url): 
    box = soup.find('div', class_='region region-content')
    volumen=box.findAll('div', class_='row text-justify')
    vol = [x.find('a').get('href')for x in volumen]
    vol=[urljoin(url_root,i) for i in vol]
    return vol

In [6]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break

Estoy en la pagina http://revistamedica.imss.gob.mx/numero_actual


In [7]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

17

In [8]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [9]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Revista Médica del Instituto Mexicano del Seguro Social'
    a='III. Medicina y Ciencias de la Salud'
    tem='Medicina Experimental, Clínica, Epidemiología, Sistemas y Servicios de Salud, Educación Médica, Historia y Filosofía de la Medicina, Ciencias Sociales y Salud, ética Médica'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo Revista']=tire
    except AttributeError:
        content_book['Titulo Revista']=None
    #area
    try:
        area=a
        content_book['Area']=a
    except AttributeError:
        content_book['Area']=None
    #tematica
    try:
        tema=tem
        content_book['Tematica']=tem
    except AttributeError:
        content_book['Tematica']=None
    #titulo articulo
    try:
        titu=s_item.find('div', id='articleTitle').get_text(strip=True)
        content_book['Titulo Articulo']=titu
    except AttributeError:
        content_book['Titulo Articulo']=None
    #resumen
    try:
        resu=s_item.find('div', id='articleAbstract').find('p')
        content_book['Resumen']=resu.find_next_sibling('p').get_text(strip=True)
    except AttributeError:
        content_book['Resumen']=None
    #abstract
    try:
        abst=s_item.find('div', id='item cover_image').get_text(strip=True)
        content_book['Abstract']=abst
    except AttributeError:
        content_book['Abstract']=None
    #Link incial
    try:
        linkart=url
        content_book['Link Articulo']=linkart
    except AttributeError:
        content_book['Link Articulo']=None
    #link articulo
    try:
        link=s_item.find('a', class_='file').get('href')
        content_book['Link PDF']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

In [10]:
list_scraper=list_scraper[0:17]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6
estas escrapeando la pag 7
estas escrapeando la pag 8
estas escrapeando la pag 9
estas escrapeando la pag 10
estas escrapeando la pag 11
estas escrapeando la pag 12
estas escrapeando la pag 13
estas escrapeando la pag 14
estas escrapeando la pag 15
estas escrapeando la pag 16


In [11]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo Revista,Area,Tematica,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
0,Revista Médica del Instituto Mexicano del Segu...,III. Medicina y Ciencias de la Salud,"Medicina Experimental, Clínica, Epidemiología,...",La variabilidad genética en la asociación escl...,La esclerosis múltiple es una enfermedad neuro...,None,http://revistamedica.imss.gob.mx/editorial/ind...,http://revistamedica.imss.gob.mx/editorial/ind...
1,Revista Médica del Instituto Mexicano del Segu...,III. Medicina y Ciencias de la Salud,"Medicina Experimental, Clínica, Epidemiología,...",Variabilidad de la hemoglobina y hematocrito d...,"Introducción:usualmente, la hemoglobina y el h...",None,http://revistamedica.imss.gob.mx/editorial/ind...,http://revistamedica.imss.gob.mx/editorial/ind...
2,Revista Médica del Instituto Mexicano del Segu...,III. Medicina y Ciencias de la Salud,"Medicina Experimental, Clínica, Epidemiología,...",Estado actual de resistencia antimicrobiana e...,"Introducción:hoy en día, la resistencia bacter...",None,http://revistamedica.imss.gob.mx/editorial/ind...,http://revistamedica.imss.gob.mx/editorial/ind...
3,Revista Médica del Instituto Mexicano del Segu...,III. Medicina y Ciencias de la Salud,"Medicina Experimental, Clínica, Epidemiología,...",Perfil clínico-epidemiológico de pacientes con...,Introducción:la paralisis periodica tirotóxica...,None,http://revistamedica.imss.gob.mx/editorial/ind...,http://revistamedica.imss.gob.mx/editorial/ind...
4,Revista Médica del Instituto Mexicano del Segu...,III. Medicina y Ciencias de la Salud,"Medicina Experimental, Clínica, Epidemiología,...",Capacitación sobre lactancia materna en embar...,Introducción:la lactancia materna inadecuada s...,None,http://revistamedica.imss.gob.mx/editorial/ind...,http://revistamedica.imss.gob.mx/editorial/ind...
5,Revista Médica del Instituto Mexicano del Segu...,III. Medicina y Ciencias de la Salud,"Medicina Experimental, Clínica, Epidemiología,...",Sintomatología cardiorrespiratoria y neuromusc...,Introducción:la COVID-19 afecta predominanteme...,None,http://revistamedica.imss.gob.mx/editorial/ind...,http://revistamedica.imss.gob.mx/editorial/ind...
6,Revista Médica del Instituto Mexicano del Segu...,III. Medicina y Ciencias de la Salud,"Medicina Experimental, Clínica, Epidemiología,...",Reinserción laboral en trabajadores con incapa...,Introducción:las lesiones de mano derivadas de...,None,http://revistamedica.imss.gob.mx/editorial/ind...,http://revistamedica.imss.gob.mx/editorial/ind...
7,Revista Médica del Instituto Mexicano del Segu...,III. Medicina y Ciencias de la Salud,"Medicina Experimental, Clínica, Epidemiología,...",Factores epidemiológicos asociados a recurren...,Introducción:la infección de vías urinarias en...,None,http://revistamedica.imss.gob.mx/editorial/ind...,http://revistamedica.imss.gob.mx/editorial/ind...
8,Revista Médica del Instituto Mexicano del Segu...,III. Medicina y Ciencias de la Salud,"Medicina Experimental, Clínica, Epidemiología,...","Actividad física, perfil bioquímico y somatomé...",Introducción:Durante la contengencia por COVID...,None,http://revistamedica.imss.gob.mx/editorial/ind...,http://revistamedica.imss.gob.mx/editorial/ind...
9,Revista Médica del Instituto Mexicano del Segu...,III. Medicina y Ciencias de la Salud,"Medicina Experimental, Clínica, Epidemiología,...",Estrategia de Follow up en la gestión de la e...,Introducción:en los países con economías emerg...,None,http://revistamedica.imss.gob.mx/editorial/ind...,http://revistamedica.imss.gob.mx/editorial/ind...


In [12]:
df_catalogo.to_csv('Revista87.csv', index=False)